# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.risk.linalg import pca, cholesky

from cvx.risk.factor import FactorModel

## An abstract Risk Model

An abstract risk model is the parent for concrete risk models. It serves as the blueprint. No instances of this class can be created.
Each risk model inherits the interface and all attributes defined in this class. 

An optimizer would get a risk model. Since all risk models share the interface it is trivial to change the risk model.

In [2]:
from abc import abstractmethod, ABC


class RiskModel(ABC):
    @abstractmethod
    def estimate_risk(self, weights, **kwargs):
        """
        Estimate the variance of a portfolio given its weights
        The weights may come as a cvxpy Variable or numpy array.
        """

## Load prices and compute returns

In [3]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

## Compute principal components

In [4]:
factors = pca(returns=returns, n_components=10)

## Create the risk model, here a FactorModel

In [5]:
model = FactorModel(assets=len(returns.columns), k=10)

# update the model parameters
model.chol.value = cholesky(factors.cov)
model.exposure.value = factors.exposure
model.idiosyncratic_risk.value = factors.idiosyncratic.std().values

# test the risk model with uniform weights
weights = pd.Series(index = prices.columns, data=0.05).values
model.estimate_risk(weights).value

8.526818368171256e-05

## RiskModel is injected into optimizer

In [6]:
w = cvx.Variable(20)
y = cvx.Variable(10)

objective = cvx.Minimize(1e3*model.estimate_risk(w, y=y))
constraints = [w >= 0, cvx.sum(w) == 1, y == model.exposure @ w]

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate_risk(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

GOOG    3.726134e-24
AAPL    1.307768e-02
FB      3.614237e-25
BABA    5.587538e-02
AMZN    3.332625e-02
GE      6.494129e-03
AMD    -1.265276e-22
WMT     4.472814e-02
BAC    -1.000186e-23
GM     -2.356836e-23
T       1.330519e-01
UAA    -5.489793e-23
SHLD   -3.594943e-22
XOM     2.498250e-01
RRC    -2.624581e-23
BBY     1.102160e-02
MA      6.236276e-02
PFE     2.407212e-01
JPM    -7.403189e-24
SBUX    1.495160e-01
dtype: float64
4.620056983380704e-05
